In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import os
from os import path
from datetime import datetime

In [2]:
driver_path = path.join(path.dirname(os.getcwd()),'drivers','chromedriver.exe')

In [3]:
driver = webdriver.Chrome(executable_path=driver_path)

In [4]:
vi_base_url = 'https://www.ville-ideale.fr/'

In [5]:
city_test = {
    'name':'puteaux',
    'postal_code': 92062
}

In [6]:
driver.get(url = vi_base_url+'{}_{}'.format(city_test['name'], city_test['postal_code']))

In [7]:
soup = BeautifulSoup(driver.page_source)

## Study on general information and notes

In [8]:
soup.title.text.split(' ')[2]

'PUTEAUX'

In [9]:
city_dimension_label = {
     'Environnement': 'environment',
     'Transports': 'transport',
     'Sécurité': 'security',
     'Santé': 'health',
     'Sports et loisirs': 'sport_entertainment',
     'Culture': 'culture',
     'Enseignement':'education',
     'Commerces': 'business',
     'Qualité de vie': 'life_quality'
 }

In [10]:
dict_cities_notes = {}

In [11]:
city_note = {}
city_note['name'] = soup.find(id='colleft').find('h1').text.split(' ')[0]
city_note['postal_code'] = soup.find(id='colleft').find('h1').text.split(' ')[1][1:-1]
general_note = soup.find(id='ng')
city_note['general_note'] = float(general_note.text.split(' ')[0].replace(',','.'))
city_note['count'] = int(soup.find(id='nobt').text.split(' ')[-2])
# city_note['ranking']
general_note_table = soup.find(id='tablonotes')
for label_note in general_note_table.find_all('tr'):
    city_note[city_dimension_label[label_note.find('th').text]] =  float(label_note.find('td').text.replace(',','.'))
city_note['update_time'] = datetime.now()

In [12]:
soup.find(id='classt').find('b').text

'24ème\xa0/\xa0213'

In [13]:
dict_cities_notes['puteaux'] = city_note

In [14]:
dict_cities_notes

{'puteaux': {'name': 'PUTEAUX',
  'postal_code': '92800',
  'general_note': 7.67,
  'count': 155,
  'environment': 6.8,
  'transport': 8.12,
  'security': 8.04,
  'health': 7.31,
  'sport_entertainment': 8.56,
  'culture': 7.9,
  'education': 6.25,
  'business': 7.56,
  'life_quality': 7.76,
  'update_time': datetime.datetime(2021, 5, 15, 23, 33, 31, 535545)}}

## Study on user reviews

In [15]:
import re

In [16]:
review_blocks = soup.find_all(class_='comm')

In [17]:
len(review_blocks)

6

In [18]:
review_block = review_blocks[0]

In [19]:
review_block.find_all('p')[0].text

'Avis posté le 02-05-2021 à 18:03Par Sam'

get edit time

In [20]:
re_search = re.search(r'\d{2}-\d{2}-\d{4}',review_block.find_all('p')[0].text)
date_text = re_search.group(0)
re_search = re.search(r'\d{2}:\d{2}',review_block.find_all('p')[0].text)
time_text = re_search.group(0)
post_time = datetime.strptime('{}-{}'.format(date_text, time_text),'%d-%m-%Y-%H:%M')

In [21]:
dimensions = [th.text for th in review_block.find_all('th')]
notes = [int(td.text) for td in review_block.find_all('td')]

In [22]:
review_block_data = {
    'time': post_time,
    'mean':float(review_block.find(class_='moyenne').text),
    **dict(zip(dimensions, notes)),
    'positive': review_block.find_all('p')[1].text,
    'negative': review_block.find_all('p')[2].text,
    'count_agree': int(review_block.find_all('p')[3].find_all('strong')[0].text),
    'count_disagree': int(review_block.find_all('p')[3].find_all('strong')[1].text)
}

In [23]:
review_block_data

{'time': datetime.datetime(2021, 5, 2, 18, 3),
 'mean': 9.38,
 'Environnement': 8,
 'Transports': 10,
 'Sécurité': 9,
 'Santé': 9,
 'Sportset loisirs': 9,
 'Culture': 9,
 'Enseignement': 7,
 'Commerces': 9,
 'Qualitéde vie': 10,
 'positive': "Les points positifs : Transport\nRER/Métro ligne 1\n\nSécurité\nPolice municipale réactive\n\nVerdure\nBois de Boulogne à proximité\nParc de l'île de Puteaux\n\nCommerces diversifiés et pour tout budget.",
 'negative': 'Les points négatifs : Ville bruyante\nDépartementale, circulation de plus en plus dense\n\nCrèche\nPeu de place en crèche\n\nQuartier frontalier à Nanterre (haut de Puteaux) à éviter\n\nVille très agréable.',
 'count_agree': 4,
 'count_disagree': 4}